In [15]:
import os, string
import ggl_img_scraper as ggl
from tqdm import tqdm
from utils import *

In [13]:
# Necessary inputs
ggl_api_key = ''
search_engine_id = 'f1aca5d66c8d4435c'

dataset_dir = '..\\data\\datasets\\smaller_birds_dataset\\'
raw_dir = dataset_dir + 'raw\\'
masks_dir = dataset_dir + 'masks\\'

num_images = 75
#'..\\data\\bird_lists\\bay_area_birds.txt'
birds_txt = '..\\data\\bird_lists\\short_bay_area_birds.txt'
db_name = ".\\bird_im_urls.db"

buffer = 5

## !!! DANGER ZONE
clear_dirs = False
remove_db = False

In [14]:
## Clear dir
if clear_dirs:
    clear_dir(raw_dir, [])
    clear_dir(masks_dir,[])
if remove_db and os.path.exists(db_name):
    os.remove(db_name)
    if os.path.exists(db_name+"-journal"):
        os.remove(db_name+"-journal")

In [16]:
# Read-in txt file of bird names
# File must be new-line delimited
birds = []
with open(birds_txt) as f:
    if f.readable() is False:
        raise FileNotFoundError("ERROR: File is not a readable file.")
    birds = f.readlines()
birds = [ string.capwords(x.strip()) for x in birds]
birds.sort()

In [12]:
# Make a directory for every bird in the list in the training and validation directories
for bird in birds:
    if not os.path.exists(raw_dir + bird + '\\'):
        os.makedirs(raw_dir + bird + '\\')
    if not os.path.exists(masks_dir + bird + '\\'):
        os.makedirs(masks_dir + bird + '\\')

In [ ]:
retry_lst = {}

In [ ]:
# Get Images for each bird from Google Images
for bird in tqdm(birds):
        search_query = f"real '{str(bird).strip()}' bird -drawing -map -cartoon -logo -baby -egg -painting -pattern -illustration -art -similar -information -creative -general -book -math -product -food -feed -help -zoologist -list -bingo -tattoo -ranch -cowboy -nest -jewelry -necklace -sports"
        save_dir = raw_dir + '\\'+bird+'\\'
    saved = ggl.google_image_download(query=search_query, save_directory=save_dir, api_key=ggl_api_key, cx=search_engine_id, n=num_images, name=bird, db_name=db_name,delay=None, mute=True)
    if (len(saved)+buffer)  < num_images:
        retry_lst[bird] = saved

In [ ]:
# Counts directories for correct number of images
count_jpg_images(raw_dir, num_images, buffer = buffer,raise_e = False)

In [ ]:
print(len(retry_lst))
print(retry_lst)

In [ ]:
# Retry getting images for brids in the list with a slightly different query
for bird in tqdm(retry_lst.keys()):
    search_query = str(bird)
    save_dir = raw_dir + '\\'+str(bird)+'\\'
    saved = ggl.google_image_download(query=search_query, save_directory=save_dir, api_key=ggl_api_key, cx=search_engine_id, n=num_images-retry_lst[bird], name=bird, db_name=db_name,delay=None, exclude_urls=retry_lst[bird])
    if (len(saved)+retry_lst[bird]+buffer) < num_images:
        print(saved)
        print(bird + " saved " + str(len(saved)+retry_lst[bird]) + " not " + str(num_images))
    

In [ ]:
input_img_paths = sorted(
    [
        os.path.join(root, file) 
        for root, _, files in os.walk(raw_dir) 
        for file in files 
        if file.lower().endswith('.jpg') and not file.startswith(".")
    ]
)

In [ ]:
## Converts Images to RGB mode
error_ims = [convert_image_mode(input_img) for input_img in input_img_paths if convert_image_mode(input_img, 'RGB') is not None]

In [ ]:
print("ERROR Removing:\n")
print(error_ims)

In [ ]:
# Removes files that couldn't convert to RGB mode
for im in error_ims:
    if os.path.exists(im):
        os.remove(im)
    else:
        print("ERROR REMOVEING: ", im)

#### REMOVE AND BAD DATA DOWNLOADED FROM SCRAPER BEFORE CONTINUING